In [2]:
!pip install simple_salesforce

In [3]:
import json
from sqlalchemy import create_engine
from datetime import date
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [4]:
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [5]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [6]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

In [7]:
#Student
student_data_df = pd.read_sql("SELECT * FROM student", conn)
student_data_df.head(5)

,ID_Student,StudentID,LastName,FirstName,MiddleName,BirthDate,Gender
0,33,25004961,Moore,Heather,Alice,None,F
1,34,25003514,Multak,Ilana,Cecille,None,F
2,35,25005833,Murillo,Jessica,Dorothy,None,F
3,36,25002589,Romanowski,Kandra,Genevieve,None,F
4,37,25007185,Hoffer,Katherine,Lynnette,None,F


In [8]:
student_data_records = student_data_df.to_dict('records')
student_data_records

[{'ID_Student': 33,
  'StudentID': '25004961',
  'LastName': 'Moore',
  'FirstName': 'Heather',
  'MiddleName': 'Alice',
  'BirthDate': None,
  'Gender': 'F'},
 {'ID_Student': 34,
  'StudentID': '25003514',
  'LastName': 'Multak',
  'FirstName': 'Ilana',
  'MiddleName': 'Cecille',
  'BirthDate': None,
  'Gender': 'F'},
 {'ID_Student': 35,
  'StudentID': '25005833',
  'LastName': 'Murillo',
  'FirstName': 'Jessica',
  'MiddleName': 'Dorothy',
  'BirthDate': None,
  'Gender': 'F'},
 {'ID_Student': 36,
  'StudentID': '25002589',
  'LastName': 'Romanowski',
  'FirstName': 'Kandra',
  'MiddleName': 'Genevieve',
  'BirthDate': None,
  'Gender': 'F'},
 {'ID_Student': 37,
  'StudentID': '25007185',
  'LastName': 'Hoffer',
  'FirstName': 'Katherine',
  'MiddleName': 'Lynnette',
  'BirthDate': None,
  'Gender': 'F'},
 {'ID_Student': 38,
  'StudentID': '25006014',
  'LastName': 'Poocharoen',
  'FirstName': 'Pariya',
  'MiddleName': 'Mariette',
  'BirthDate': None,
  'Gender': 'F'},
 {'ID_Student'

In [9]:
for rec in student_data_records:

    record = {
        'ID_Student__c': rec['ID_Student'],
        'StudentID__c': rec['StudentID'],
        'LastName__c': rec['LastName'],
        'FirstName__c': rec['FirstName'],
        'MiddleName__c': rec['MiddleName'],
        'BirthDate__c': rec['BirthDate'],
        'Gender__c': rec['Gender']
    }
    
    try:
        sf.Student__c.create(record)
    except Exception as e:
        print(e)

In [10]:
student_lookup_list = []

data = sf.query_all_iter("SELECT ID_Student__c, Name FROM Student__c")
for row in data:
    rec = {
        'ID_Student': row['ID_Student__c'],
        'StudentSF': row['Name']
    }
    student_lookup_list.append(rec)

In [11]:
student_lookup_df = pd.DataFrame(student_lookup_list)
student_lookup_df.head(5)

,ID_Student,StudentSF
0,37.0,a093h000000kB7r
1,53.0,a093h000000kB99
2,33.0,a093h000000kB7X
3,45.0,a093h000000kB8V
4,60.0,a093h000000kB9i


In [12]:
class_lookup_list = []

data = sf.query_all_iter("SELECT ID_Class__c, Name FROM Class__c")
for row in data:
    rec = {
        'ID_Class': row['ID_Class__c'],
        'ClassSF': row['Name']
    }
    class_lookup_list.append(rec)

In [13]:
class_lookup_df = pd.DataFrame(class_lookup_list)
class_lookup_df

""


In [14]:
#Classparticipant
query = '''
    SELECT
        cp.*
        
    FROM
        classparticipant cp
        
'''
classparticipant_data_df = pd.read_sql(query, conn)
classparticipant_data_df.head()

,ID_ClassParticipant,ID_Student,ID_Class,StartDate,EndDate
0,1,33,1,2020-03-16,None
1,2,34,1,2020-03-16,None
2,3,35,1,2020-03-16,None
3,4,62,1,2020-03-16,None
4,5,36,1,2020-03-16,None


In [15]:
classparticipant_data_df = pd.merge(classparticipant_data_df, student_lookup_df, how='left')
classparticipant_data_df.head()

,ID_ClassParticipant,ID_Student,ID_Class,StartDate,EndDate,StudentSF
0,1,33,1,2020-03-16,None,a093h000000kB7X
1,2,34,1,2020-03-16,None,a093h000000kB7c
2,3,35,1,2020-03-16,None,a093h000000kB7h
3,4,62,1,2020-03-16,None,a093h000000kB9s
4,5,36,1,2020-03-16,None,a093h000000kB7m


In [16]:
classparticipant_data_df = pd.merge(classparticipant_data_df, class_lookup_df, how='left')
classparticipant_data_df.head()

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [17]:
classparticipant_data_records = classparticipant_data_df.to_dict(orient='records')
classparticipant_data_records

[{'ID_ClassParticipant': 1,
  'ID_Student': 33,
  'ID_Class': 1,
  'StartDate': datetime.date(2020, 3, 16),
  'EndDate': None,
  'StudentSF': 'a093h000000kB7X'},
 {'ID_ClassParticipant': 2,
  'ID_Student': 34,
  'ID_Class': 1,
  'StartDate': datetime.date(2020, 3, 16),
  'EndDate': None,
  'StudentSF': 'a093h000000kB7c'},
 {'ID_ClassParticipant': 3,
  'ID_Student': 35,
  'ID_Class': 1,
  'StartDate': datetime.date(2020, 3, 16),
  'EndDate': None,
  'StudentSF': 'a093h000000kB7h'},
 {'ID_ClassParticipant': 4,
  'ID_Student': 62,
  'ID_Class': 1,
  'StartDate': datetime.date(2020, 3, 16),
  'EndDate': None,
  'StudentSF': 'a093h000000kB9s'},
 {'ID_ClassParticipant': 5,
  'ID_Student': 36,
  'ID_Class': 1,
  'StartDate': datetime.date(2020, 3, 16),
  'EndDate': None,
  'StudentSF': 'a093h000000kB7m'},
 {'ID_ClassParticipant': 6,
  'ID_Student': 37,
  'ID_Class': 1,
  'StartDate': datetime.date(2020, 3, 16),
  'EndDate': None,
  'StudentSF': 'a093h000000kB7r'},
 {'ID_ClassParticipant': 7,


In [18]:
for rec in classparticipant_data_records:
 
    record = {
        'ID_ClassParticipant__c': rec['ID_ClassParticipant'],
        'ID_Student__c': rec['ID_Student'],
        'ID_Class__c': rec['ID_Class'],
        'StartDate__c': rec['StartDate'].isoformat(),
        'EndDate__c': rec['EndDate'],
        'Student_Name__c': rec['StudentSF'],
        'Class_Name__c': rec['ClassSF']
    }
    
    try:
        sf.Class_Participant__c.create(record)
    except Exception as e:
        print(e)

KeyError: 'ClassSF'